In [18]:
# Some steps required until these Python modules are properly installed...
#
import sys
import shutil
import glob
%pylab inline
import pandas as pd
import os
#
sys.path.append('../../../Modules')
sys.path.append('../../../../veneer-py')
# Get the Source scripting components (veneer) and GBR specific stuff
#
import veneer
import gbr

Populating the interactive namespace from numpy and matplotlib


In [19]:
### Declare variables of interest
#
Scenario_Directory = 'D:/a_sc_p2r_b_rc9/Model_runs/Burdekin Rebuild 2014'
Relevant_Scenarios = ['BU_RC9_BL_3_Hill_1','BU_RC9_BL_3_Hill_2','BU_RC9_BL_3_Hill_3','BU_RC9_BL_3_Hill_4']
Spatial_Output = 'D:/a_sc_p2r_b_rc9/Model_runs/Burdekin Rebuild 2014/Hill_Scenarios_Concentration.csv'

Burdekin_intSCFU = "d:/a_sc_p2r_b_rc9/Configuration/BU_intSCFU.csv"

# Warning, You will have to manually add the retrivial and joining of the scenarios, till someone writes a loop,
# See bottom cells for a better explanaton


In [20]:
# Get an example scenario results set
#
gbr.init(Scenario_Directory)
results = gbr.Results('BU_RC9_BL_3_Hill_1')
Raw_Results = results.get('RawResults')
Raw_Results = Raw_Results.reset_index()
#
# Filter out zero values
#
Raw_Results = Raw_Results[Raw_Results>0].dropna()
#
# Filter fine sediment hillslope surface soil load 
# 
Hillslope_surface_soil = Raw_Results[Raw_Results.Constituent=='Sediment - Fine']
Hillslope_surface_soil = Hillslope_surface_soil[Hillslope_surface_soil.ModelElementType=='Catchment']
Hillslope_surface_soil = Hillslope_surface_soil[Hillslope_surface_soil.BudgetElement=='Hillslope surface soil']
#
# Calculate fine sediment hillslope surface soil load Load (t)
#
Hillslope_surface_soil.reset_index(inplace=True)
Hillslope_surface_soil['Load_t'] = (Hillslope_surface_soil['Total_Load_in_Kg'])/1000
Hillslope_surface_soil = Hillslope_surface_soil.set_index(['ModelElement','FU'])
#
# Remove superfluous columns
#
Hillslope_surface_soil = Hillslope_surface_soil[['Constituent','ModelElementType','Process','Load_t']]
#
# Rename and Set index by Catchment FU columns
#
Hillslope_surface_soil.reset_index(inplace=True)
Hillslope_surface_soil = Hillslope_surface_soil.rename(columns={'ModelElement': 'Catchment'})
Hillslope_surface_soil = Hillslope_surface_soil.set_index(['Catchment','FU'])
Hillslope_surface_soil [0:1]

,,Constituent,ModelElementType,Process,Load_t
Catchment,FU,,,,
SC #1608,Grazing Forested,Sediment - Fine,Catchment,Supply,9894.917536


In [21]:
# Get the 'fuAreasTable' for area and volume calculations
Area = results.get('fuAreasTable')
Area = Area.reset_index()
# Remove zero values from 'fuAreasTable', in order to remove redundant scenario place holder FU
Area = Area[Area>0].dropna()
# Calc area in ha
# Set index as Catchment and FU 
Area = Area.set_index(['Catchment','FU'])
Area [0:1]

,,Area
Catchment,FU,
SC #1608,Water,4.386668e+06


In [22]:
# Get the climateTable as we are chasing hydro results
#
Climate = results.get('climateTable')
Climate = Climate.reset_index()
# Remove zero values from climateTable, in order to remove redundant scenario place holder FU
#
Climate = Climate[Climate>0].dropna()
Climate [0:1]
# Merge climate and area table together to allow a calculation of total volume 
#
Area = Area.reset_index()
Climate_Area = pd.merge(Climate,Area, on=['Catchment','FU'])
Climate_Area [0:1]
# Calculate a 'Total_Volume_m3' column - Total_Volume_m3 = Depth (m) * Area (m2) 
#
Climate_Area['Total_Volume_m3'] = Climate_Area['Depth_m'] * Climate_Area['Area']
Climate_Volume = Climate_Area
# Pivot Catchment and FU as index
#
Climate_Volume = pd.pivot_table(Climate_Volume, index = ['Catchment','FU'], columns = 'Element', values = 'Total_Volume_m3', aggfunc=np.sum)
Climate_Volume.reset_index()
Climate_Volume [0:3]
# Calculate Total Runoff column - Baseflow + Quickflow
#
Climate_Volume['Total_Runoff'] = Climate_Volume['Baseflow'] + Climate_Volume['Runoff (QuickFlow)']
Climate_Volume = Climate_Volume.reset_index()
Climate_Volume = Climate_Volume.set_index(['Catchment','FU'])
Climate_Volume [0:1]

,Element,Actual ET,Baseflow,Rainfall,Runoff (QuickFlow),Total_Runoff
Catchment,FU,,,,,
SC #10,Conservation,7.808681e+08,5.545803e+07,1.143753e+09,3.041031e+08,3.595611e+08


In [23]:
# Calculate Concentration
#
# Merge with load to facilitate Concentration Calculation
Load_and_Volume = Hillslope_surface_soil.join(Climate_Volume)
Load_and_Volume [0:1]

,,Constituent,ModelElementType,Process,Load_t,Actual ET,Baseflow,Rainfall,Runoff (QuickFlow),Total_Runoff
Catchment,FU,,,,,,,,,
SC #1608,Grazing Forested,Sediment - Fine,Catchment,Supply,9894.917536,2.585894e+09,3.545974e+08,3.661731e+09,6.730334e+08,1.027631e+09


In [24]:
# Calculate fine sediment - hillslope surface soil load (g/L)
#
Load_and_Volume["FS_G_L"] = (Load_and_Volume.Load_t*1000000)/(Load_and_Volume.Total_Runoff*1000)
Load_and_Volume [0:1]
#  Remove superfluous columns and set Catchment FU cols as index 
#
Conc = Load_and_Volume.drop(['Constituent','ModelElementType','Process','Load_t','Actual ET','Baseflow','Rainfall','Runoff (QuickFlow)','Total_Runoff'], axis=1)
Conc [0:1]

,,FS_G_L
Catchment,FU,
SC #1608,Grazing Forested,0.009629


In [25]:
# Translating it to a reusable function
# 
def ScenResults(results):
    Raw_Results = results.get('RawResults')
    Raw_Results = Raw_Results.reset_index()
    #
    # Filter out zero values
    #
    Raw_Results = Raw_Results[Raw_Results>0].dropna()
    #
    # Filter fine sediment hillslope surface soil load 
    # 
    Hillslope_surface_soil = Raw_Results[Raw_Results.Constituent=='Sediment - Fine']
    Hillslope_surface_soil = Hillslope_surface_soil[Hillslope_surface_soil.ModelElementType=='Catchment']
    Hillslope_surface_soil = Hillslope_surface_soil[Hillslope_surface_soil.BudgetElement=='Hillslope surface soil']
    #
    # Calculate fine sediment hillslope surface soil load Load (t)
    #
    Hillslope_surface_soil.reset_index(inplace=True)
    Hillslope_surface_soil['Load_t'] = (Hillslope_surface_soil['Total_Load_in_Kg'])/1000
    Hillslope_surface_soil = Hillslope_surface_soil.set_index(['ModelElement','FU'])
    #
    # Remove superfluous columns
    #
    Hillslope_surface_soil = Hillslope_surface_soil[['Constituent','ModelElementType','Process','Load_t']]
    #
    # Rename and Set index by Catchment FU columns
    #
    Hillslope_surface_soil.reset_index(inplace=True)
    Hillslope_surface_soil = Hillslope_surface_soil.rename(columns={'ModelElement': 'Catchment'})
    Hillslope_surface_soil = Hillslope_surface_soil.set_index(['Catchment','FU'])
    Hillslope_surface_soil [0:1]
    # Get the 'fuAreasTable' for area and volume calculations
    Area = results.get('fuAreasTable')
    Area = Area.reset_index()
    # Remove zero values from 'fuAreasTable', in order to remove redundant scenario place holder FU
    Area = Area[Area>0].dropna()
    # Set index as Catchment and FU 
    Area = Area.set_index(['Catchment','FU'])
    Area [0:1]
    # Get the climateTable as we are chasing hydro results
    #
    Climate = results.get('climateTable')
    Climate = Climate.reset_index()
    # Remove zero values from climateTable, in order to remove redundant scenario place holder FU
    #
    Climate = Climate[Climate>0].dropna()
    Climate [0:1]
    # Merge climate and area table together to allow a calculation of total volume 
    #
    Area = Area.reset_index()
    Climate_Area = pd.merge(Climate,Area, on=['Catchment','FU'])
    Climate_Area [0:1]
    # Calculate a 'Total_Volume_m3' column - Total_Volume_m3 = Depth (m) * Area (m2) 
    #
    Climate_Area['Total_Volume_m3'] = Climate_Area['Depth_m'] * Climate_Area['Area']
    Climate_Volume = Climate_Area
    # Pivot Catchment and FU as index
    #
    Climate_Volume = pd.pivot_table(Climate_Volume, index = ['Catchment','FU'], columns = 'Element', values = 'Total_Volume_m3', aggfunc=np.sum)
    Climate_Volume.reset_index()
    Climate_Volume [0:3]
    # Calculate Total Runoff column - Baseflow + Quickflow
    #
    Climate_Volume['Total_Runoff'] = Climate_Volume['Baseflow'] + Climate_Volume['Runoff (QuickFlow)']
    Climate_Volume = Climate_Volume.reset_index()
    Climate_Volume = Climate_Volume.set_index(['Catchment','FU'])
    Climate_Volume [0:1]
    # Calculate Concentration
    #
    # Merge with load to facilitate Concentration Calculation
    Load_and_Volume = Hillslope_surface_soil.join(Climate_Volume)
    # Calculate fine sediment - hillslope surface soil load (g/L)
    #
    Load_and_Volume["FS_G_L"] = (Load_and_Volume.Load_t*1000000)/(Load_and_Volume.Total_Runoff*1000)
    Load_and_Volume [0:1]
    #  Remove superfluous columns and set Catchment FU cols as index 
    #
    Conc = Load_and_Volume.drop(['Constituent','ModelElementType','Process','Load_t','Actual ET','Baseflow','Rainfall','Runoff (QuickFlow)','Total_Runoff'], axis=1)
    Conc [0:1]
    return Conc

In [26]:
# Ok - Need to work out how to loop the following
results = gbr.Results('BU_RC9_BL_3_Hill_1')
MRY_1 = ScenResults(results)
MRY_1 [0:1]

,,FS_G_L
Catchment,FU,
SC #1608,Grazing Forested,0.009629


In [27]:
# Looping through the Relevant Scenarios, calculating conc using the function outlined above
Hillslope_Sceanrios_FS_G_L = dict()
for Scen in Relevant_Scenarios:
    gbr.init(Scenario_Directory)
    gbr.available()
    results = gbr.Results(Scen)
    Hillslope_Sceanrios_FS_G_L[Scen] = ScenResults(results)

In [28]:
Scen1 = Hillslope_Sceanrios_FS_G_L['BU_RC9_BL_3_Hill_1']
Scen1.rename(columns={'FS_G_L':'FS_G_L_1'},inplace=True)
Scen1 [0:1]

,,FS_G_L_1
Catchment,FU,
SC #1608,Grazing Forested,0.009629


In [29]:
Scen2 = Hillslope_Sceanrios_FS_G_L['BU_RC9_BL_3_Hill_2']
Scen2.rename(columns={'FS_G_L':'FS_G_L_2'},inplace=True)
Scen2 [0:1]

,,FS_G_L_2
Catchment,FU,
SC #1608,Grazing Forested,0.006427


In [30]:
Scen3 = Hillslope_Sceanrios_FS_G_L['BU_RC9_BL_3_Hill_3']
Scen3.rename(columns={'FS_G_L':'FS_G_L_3'},inplace=True)
Scen3 [0:1]

,,FS_G_L_3
Catchment,FU,
SC #1608,Grazing Forested,0.002613


In [31]:
All_Scen = pd.concat([Scen1, Scen2, Scen3], axis=1, join='inner')
All_Scen [0:1]

,,FS_G_L_1,FS_G_L_2,FS_G_L_3
Catchment,FU,,,
SC #1608,Grazing Forested,0.009629,0.006427,0.002613


In [32]:
# Read in intsFu table, remove superfluous columns and set Catchment FU cols as index 
#
BU_intSCFU = pd.read_csv(Burdekin_intSCFU, error_bad_lines=False)
BU_intSCFU.rename(columns={'IntFUs':'FU','IntSCs':'Catchment'},inplace=True)
BU_intSCFU = BU_intSCFU.drop(['IntSCFU', 'FID'], axis=1)
BU_intSCFU.set_index(['Catchment','FU'],inplace=True)
BU_intSCFU [0:1]

,,IntSCFU_S
Catchment,FU,
SC #494,Grazing Forested,0


In [33]:
## Join with Scenario
#
Scenario_Table = BU_intSCFU.join(All_Scen)
Scenario_Table [0:1]

,,IntSCFU_S,FS_G_L_1,FS_G_L_2,FS_G_L_3
Catchment,FU,,,,
SC #494,Grazing Forested,0,0.084077,0.058495,0.050255


In [34]:
# Export To CSV
#
Scenario_Table.to_csv(Spatial_Output)